# DSC180A - Q1 Project

Henry Luu B09

In [ ]:
import csv
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt

## Liar Liar Dataset

In [ ]:
columns = ['ID', 'Label', 'Statement', 'Subject(s)', 'Speaker','Speaker\'s Job Title', 'State Info', 'Party Affiliation','Barely True', 'False', 'Half True', 'Mostly True', 'Pants on Fire','Context']
df = pd.read_csv('train.tsv', delimiter='\t', header = None, quoting=csv.QUOTE_NONE)
df.head()

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
df[1].unique()

In [ ]:
#distribution of labels
label_counts = df[1].value_counts()
plt.figure(figsize=(8, 6))
plt.bar(label_counts.index, label_counts.values)
plt.title('Distribution of Veracity Labels')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()

In [ ]:
X = df.iloc[:, 8:-1] 
y = df[1]         

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)

In [ ]:
y_train_pred = clf.predict(X_train)
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f"Train Accuracy: {train_accuracy:.2f}")


In [ ]:
y_pred = clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

In [ ]:
plt.figure(figsize=(100, 100))
plot_tree(clf, filled=True, feature_names=X.columns, class_names=clf.classes_)
plt.show()

# Word embeddings


In [ ]:
import sys
import site
user_site_packages = site.USER_SITE
sys.path.append(user_site_packages)
print(sys.path)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
!pip install --user transformers

In [ ]:
import transformers
from transformers import BertTokenizer, BertModel

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
statements = df[2].tolist()
labels = df[1].tolist()

In [ ]:
#tokenizeing the statements
tokenized_statements = [tokenizer(statement, return_tensors="pt", truncation=True, padding=True) for statement in statements]

In [ ]:
import torch

# Use GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Move tokenized statements to GPU
tokenized_statements_gpu = [inputs.to(device) for inputs in tokenized_statements]

# Extract BERT embeddings
with torch.no_grad():
    model.eval()
    statement_embeddings = [model(**inputs).last_hidden_state.mean(dim=1).cpu().numpy() for inputs in tokenized_statements_gpu]

In [ ]:
#flatten
X_embeddings = np.vstack(statement_embeddings)
X_train, X_test, y_train, y_test = train_test_split(X_embeddings, labels, test_size=0.2, random_state=42)


In [ ]:
#combine the truth counts with the embeddings
X = df.iloc[:, 8:-1] 
X_embeddings_df = pd.DataFrame(X_embeddings, columns=[f"embedding_{i}" for i in range(X_embeddings.shape[1])])
X_combined = pd.concat([X, X_embeddings_df], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X_combined, labels, test_size=0.2, random_state=42)

In [ ]:
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)
print(classification_report(y_test, y_pred))